In [1]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, LearningRateScheduler
from keras import backend as K
from keras.callbacks import History, ModelCheckpoint
from sklearn.model_selection import train_test_split
from ssd300 import build_SSD300
from parser import Parser
import matplotlib.pyplot as plt
import os
from Decoder import Decoder
from Encoder import Encoder

Using TensorFlow backend.


In [2]:
# Configuration of the model: 
input_shape=(300, 300, 3)
numClasses = 10
iou_thres=0.5 # for default and gt matching
nms_thres=0.45 # IoU threshold for non-maximal suppression
score_thres=0.01 # threshold for classification scores
top_k=200 # the maximum number of predictions kept per image
min_scale=0.2 # the smallest scale of the feature map
max_scale=0.9 # the largest scale of the feature map
aspect_ratios=[0.5, 1, 2] # aspect ratios of the default boxes to be generated
n_predictions=6 # the number of prediction blocks
prediction_size=[37, 18, 10, 5, 3, 1] # sizes of feature maps at each level

### Parse Input: 


In [3]:
data_dir = "/Users/tranle/mscoco"
training_data = "val2017"
# Initialize a parser object
parser = Parser(data_dir, training_data)

# Load images and annotations for the image
# For now, we load only 10 first classes and images are resize to (300,300,3) 
# for training purposes

X, Y = parser.load_data()


loading annotations into memory...
Done (t=0.51s)
creating index...
index created!


### Ecode Parsed Input: 
In this step, we perform bipartite matching and multi-matching algorithm to match default boxes generated with the ground-truth labels of our training data. 

In [4]:
# Initialize an Ecoder object
encode = Encoder(y_truth=Y, 
                numClasses=numClasses, 
                iou_thres=iou_thres,
                min_scale=min_scale, 
                max_scale=max_scale, 
                aspect_ratios=aspect_ratios, 
                n_predictions=n_predictions, 
                prediction_size=prediction_size)

# Encode the labels and ground-truth boxes of the training images
Y = encode.get_encoded_data()


NameError: name 'aspect_ratio' is not defined

In [ ]:
# Build the SSD model
K.clear_session() # Clear previous session

# Build the model
model = build_SSD300(input_shape=input_shape, 
                  numClasses=numClasses, 
                  mode='training', 
                  min_scale=min_scale, 
                  max_scale=max_scale, 
                  aspect_ratios=aspect_ratios, 
                  iou_thres=iou_thres,
                  nms_thres=nms_thres, 
                  score_thres=score_thres, 
                  top_k=top_k,
                  n_predictions=n_predictions)

# Instantiate the Adam optimizer for the model
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# Compile the model
model.compile(optimizer=adam, loss=loss_function, 
             metrics=['accuracy'])



### Create checkpoints to store learned weights: 


In [ ]:
# Path to store learn weights
path = os.getcwd()

checkpoints = ModelCheckpoint(filepath='weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
                             monitor='val_loss'
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='min', 
                             period=1)

early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0.0,
                               patience=10,
                               verbose=1)

csv_logger = CSVLogger(filename='training_log.csv',
                       separator=',',
                       append=True)

log_dir = path + "/logs"

tensorboard = TensorBoard(log_dir=log_dir, 
                          histogram_freq=0, 
                          batch_size=32, 
                          write_graph=True, 
                          write_grads=True, 
                          write_images=True, 
                          update_freq='epoch')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                                         factor=0.2,
                                         patience=5, 
                                         min_lr=0.001)

callbacks = [checkpoints, 
            early_stopping, 
            csv_logger,
            tensorboard, 
            reduce_lr]


### Train the model: 


In [ ]:
# Set training parameters
batch_size = 16
initial_epoch = 0
total_epochs = 5
validation_split = 0.2
steps_per_epoch = 150

history = model.fit(x=X, 
                    y=Y, 
                    batch_size=batch_size,
                    epochs=total_epochs,
                    verbose=0, 
                    callbacks=callbacks,
                    validation_split=validation_slit, 
                    shuffle=True,
                    initial_epoch=initial_epoch,
                    steps_per_epoch=steps_per_epoch,
                    validation_freq=1)


### Training Visualization

In [ ]:
plt.figure(figsize=(20, 12))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val-loss'], label='Validation Loss')
plt.legend(loc='best')

### Predictions